# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [1]:
import sqlite3

In [2]:
con = sqlite3.connect("./data/Chinook_Sqlite.sqlite")
cur = con.cursor()
cur.execute('''
SELECT firstname, lastname FROM Customer WHERE country = ?
''',['Canada'])

# Проитерироваться по курсору
# .fetchone()
# .fetchmany(n)
# .fetchall()
result = cur.fetchmany(5)
print(result)

[('François', 'Tremblay'), ('Mark', 'Philips'), ('Jennifer', 'Peterson'), ('Robert', 'Brown'), ('Edward', 'Francis')]


2. Найти и вывести на экран названия всех альбомов группы Accept

In [3]:


cur = con.cursor()


cur.execute('''
SELECT Title FROM Artist  t1 INNER JOIN Album t2 ON t1.ArtistId = t2.ArtistId WHERE t1.Name = ?
''',["Accept"])

# .fetchall()
for item in cur.fetchall():
    print(item[0])

Balls to the Wall
Restless and Wild


3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [4]:
con = sqlite3.connect('test3.db')
cur = con.cursor()

cur.execute('''
CREATE TABLE Student(id INTEGER, name VARCHAR(50))
''')

cur.execute('''
INSERT INTO Student(id , name) VALUES (1, 'Nikita'),('2','Robot')
''')
con.commit()

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [5]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [6]:
cur.execute('''
CREATE TABLE recipe (
	id	INTEGER,
	name	TEXT NOT NULL,
	minutes	REAL,
	submitted	INTEGER,
	description	TEXT,
	n_ingredients	INTEGER,
	PRIMARY KEY(id AUTOINCREMENT)
);
''')

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [7]:
cur.execute('''
CREATE TABLE review (
	id	INTEGER,
	user_id	INTEGER NOT NULL,
	recipe_id	INTEGER NOT NULL,
	date	TEXT,
	rating	REAL,
	review	TEXT,
	PRIMARY KEY(id AUTOINCREMENT),
	FOREIGN KEY(recipe_id) REFERENCES Recipe(id)
);
''')

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [8]:
import pandas as pd

In [9]:
recipe = pd.read_csv('./data/recipes_sample_with_tags_ingredients.csv', sep=',')

for i, row in recipe.iterrows():
    data = row.loc[['name', 'minutes', 'submitted', 'description', 'n_ingredients']]
    cur.execute(
        '''
        INSERT INTO
        recipe (name, minutes, submitted, description, n_ingredients)
        VALUES(?, ?, ?, ?, ?)
    ''',data
    )
con.commit()

In [10]:
reviews = pd.read_csv('./data/reviews_sample.csv', sep=',')
for i, row in reviews.iterrows():
    data = row.loc[['user_id', 'recipe_id', 'date', 'rating', 'review']]
    cur.execute(
        '''
        INSERT INTO review
         (user_id, recipe_id, date, rating, review)
        VALUES(?, ?, ?, ?, ?)
    ''',
        data,
    )
con.commit()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [11]:
res = cur.execute('''
SELECT *
FROM recipe
WHERE n_ingredients = 10
''')
for item in res.fetchmany(5):
    print(item)

(14, 'blepandekager   danish   apple pancakes', 50.0, '2013-07-08', "this recipe has been posted here for play in zwt9 - scandinavia.  this recipe was found at website: mindspring.com - christian's danish recipes.", 10)
(22, 'kelly s creamy cheddar pea salad', 20.0, '2005-06-09', "i'm not a big fan of peas, but like them in things. this salad is one of my favorites. happy to share!", 10)
(25, 'middle eastern   twice baked potatoes', 110.0, '2005-05-02', 'a delicious yet low-fat twice-baked potato side dish with a blend of classic middle eastern flavours: chickpeas, cumin and coriander.  serve with roasts or with your favourite middle eastern main course dish.  adapted from "practical cookery: low fat".  these potatoes could also be served on their own as a light vegetarian meal. in view of the comments by reviewers about the potatoes being a bit dry, i have added a tablespoon of tahini. i have also added some garlic. i\'m not sure how the recipe escaped having garlic in it when i first

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [12]:
res = cur.execute('''
SELECT name
FROM recipe
WHERE minutes = (
    SELECT MAX(minutes)
    FROM recipe
)
''')

print(res.fetchone())

('blueberry liqueur',)


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [ ]:
recipe_id = input("Введите id рецепта")

res = cur.execute('''
SELECT name
FROM recipe
WHERE id = ?
''', (recipe_id,))

recipe = res.fetchone()
print(recipe[0]) if recipe else print("Рецепт не был найден!")

8. Найдите кол-во отзывов с рейтингом 5.

In [ ]:
res = cur.execute('''
SELECT COUNT(id)
FROM review
WHERE rating = 5
''')

print(res.fetchone()[0])

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [ ]:
res = cur.execute('''
SELECT COUNT(*) OVER ()
FROM recipe rec
LEFT JOIN review rev
ON rec.id = rev.recipe_id
GROUP BY rec.id
HAVING 0 = COUNT(rev.rating < 4)
LIMIT 1
''')

print(res.fetchone()[0])

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [ ]:
res = cur.execute('''
SELECT COUNT(id)
FROM recipe
WHERE submitted LIKE '2010%' AND minutes >= 15
''')

print(res.fetchone()[0])

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [ ]:
res = cur.execute('''
SELECT rec.id, rec.name, rev.user_id, rev.date, rev.rating
FROM recipe rec
LEFT JOIN review rev
ON rec.id = rev.recipe_id
WHERE rec.n_ingredients >= 3
ORDER BY rec.id
''')

print(res.fetchall())

In [ ]:
cur.close()
con.close()
